# Multicellular coordination networks

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedGroupKFold
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import scanpy as sc
import pandas as pd
import networkx as nx
import os

## Load normalized data

In [2]:
features = sc.read_h5ad("../../data/top_features_class_pT.h5ad")

/home/loan/miniforge3/envs/pam-keras3/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


## Prepare cross-validation

In [3]:
cell_table = pd.read_csv("../../data/cell_table_with_types_stage.csv", 
                         index_col = 0)

/tmp/ipykernel_199692/1000358550.py:1: DtypeWarning: Columns (39,40,41,42,43,44,45,46,48,49,52,53,54,55,56,57,58,59,60,61,62,63,65,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cell_table = pd.read_csv("../../data/cell_table_with_types_stage.csv",


In [4]:
metab_markers = ['CA9', 'CD98', 'CytC', 'MCT1', 'ASCT2', 'LDH', 'GS', 'GLS', 'ATP5A', 'CS', 'PKM2', 'GLUT1', 'ARG1', 'CPT1A', 'Ki67']

# Only metabolic markers for cancer/epithelial cells
df = cell_table.loc[cell_table.consensus == "Cancer_cell",metab_markers] 
meta = cell_table.loc[cell_table.consensus == "Cancer_cell",["Stage","fov"]]

# A few FOVs show only few epithelial cells and might only add noise to the analysis
sparse_epi_fovs = meta.fov.value_counts()[meta.fov.value_counts() <= 20].index
df = df.loc[~meta.fov.isin(sparse_epi_fovs)]
meta = meta.loc[~meta.fov.isin(sparse_epi_fovs)]

# Only keep well-annotated stages
epithelial_subset = meta["Stage"].isin(["pT1", "pT2", "pT3", "pT4"]).values

In [5]:
# Define which fovs will be held out for validation (outer loop)
fov_stage_table = meta.loc[epithelial_subset].drop_duplicates().reset_index(drop=True)
fov_inner, fov_val, y_inner, y_val = train_test_split(
    fov_stage_table["fov"], fov_stage_table["Stage"], test_size=0.2, random_state=0, stratify=fov_stage_table["Stage"])
meta["inner"] = meta["fov"].isin(fov_inner)
# For sanity check, the following should consistently be used as validation set
" ".join(fov_val.sort_values())

'A1a A1f A1h A2g A2i A2q A2r A3m A4e A4n A5a A5q A6b A6c A6g A6m A6p A6q A6r A7f A7p A8a A8m A9o A9q B1h B1k B2a B2b B2k B2o B3c B3g B4b B4g B4m B6i B6q B7c B7p B7r B8b B8d B8g B8i B8l B9c B9d B9h B9m B9n B9o C1i C1l C2a C2f C2k C3c C3h C4a C4g C4k C5a C5k C5l C6d C8a C8h D1i D1l D1m D2e D2k D4c D5b D5d D5h D5k D5l D6c D6k D7a D8d D8h'

In [6]:
n_splits = 4

cv_folds = StratifiedGroupKFold(n_splits=n_splits)
for train, test in cv_folds.split(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
                                  meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
                                  groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]):
    # All stages should be present in both train and test
    assert len(meta.loc[epithelial_subset].loc[meta["inner"]].iloc[test].groupby("fov")["Stage"].first().unique()) == 4
    assert len(meta.loc[epithelial_subset].loc[meta["inner"]].iloc[train].groupby("fov")["Stage"].first().unique()) == 4
    # Print the first test FOVs
    print(meta.loc[epithelial_subset].loc[meta["inner"]].iloc[test].groupby("fov").first().index[:5]) 

Index(['A1e', 'A1n', 'A3b', 'A3d', 'A3e'], dtype='object', name='fov')
Index(['A1c', 'A1m', 'A1o', 'A1p', 'A1q'], dtype='object', name='fov')
Index(['A1l', 'A1r', 'A2b', 'A2c', 'A2l'], dtype='object', name='fov')
Index(['A1d', 'A1i', 'A1k', 'A2a', 'A2d'], dtype='object', name='fov')


We can split the data at the cell level to allow applications that involve processing single cells. The grouping ensures that the data is split per FOV without contamination (i.e. cells from a given FOV present both in training and testing).

In [7]:
meta_per_fov = meta.loc[epithelial_subset].groupby("fov").first()

In [8]:
df_per_fov = df.copy()
df_per_fov["fov"] = meta["fov"] 
df_per_fov = df_per_fov.loc[epithelial_subset].groupby("fov").mean()
meta_per_fov = meta.loc[epithelial_subset].groupby("fov").first()

In [9]:
def conv_traintest_cells_to_fov():
    for train, test in cv_folds.split(df.loc[epithelial_subset].loc[meta.loc[epithelial_subset]["inner"]],
                                    meta.loc[epithelial_subset].loc[meta["inner"]]["Stage"],
                                    groups=meta.loc[epithelial_subset].loc[meta["inner"]]["fov"]):
        test_fovs = meta.loc[epithelial_subset].loc[meta["inner"]].iloc[test].groupby("fov").first().index
        train_fovs = meta.loc[epithelial_subset].loc[meta["inner"]].iloc[train].groupby("fov").first().index
        test_fovs_ind = np.where(meta_per_fov.loc[meta_per_fov["inner"]].index.isin(test_fovs))[0]
        train_fovs_ind = np.where(meta_per_fov.loc[meta_per_fov["inner"]].index.isin(train_fovs))[0]
        yield (train_fovs_ind, test_fovs_ind)

# Should work as the output of `split` method: two arrays of indices
cv_folds_fov = [x for x in conv_traintest_cells_to_fov()]

for train, test in cv_folds_fov:
    # All stages should be present in both train and test
    assert len(meta_per_fov.loc[meta_per_fov["inner"]].iloc[test]["Stage"].unique()) == 4
    assert len(meta_per_fov.loc[meta_per_fov["inner"]].iloc[train]["Stage"].unique()) == 4
    # Print the first test FOVs
    print(meta_per_fov.loc[meta_per_fov["inner"]].iloc[test].index[:5])

Index(['A1e', 'A1n', 'A3b', 'A3d', 'A3e'], dtype='object', name='fov')
Index(['A1c', 'A1m', 'A1o', 'A1p', 'A1q'], dtype='object', name='fov')
Index(['A1l', 'A1r', 'A2b', 'A2c', 'A2l'], dtype='object', name='fov')
Index(['A1d', 'A1i', 'A1k', 'A2a', 'A2d'], dtype='object', name='fov')


In [10]:
le = LabelEncoder()
le.fit(meta_per_fov.loc[meta_per_fov["inner"]]["Stage"])

LabelEncoder()

## Test prediction

Note on missing values: Some of the samples only had few or none of some cell types, resulting in missing values in the table. Inputation for prediction would not be very helpful, as it is likely that all the values for a cell type are missing for a patient, and thus predictions would be compared to all-inferred values. It seems better to simply exclude these specific patient + cell-type combinations from the respective testing / validation sets.

In [11]:
ref_ftype = "Metabolism"
features_subset = features[meta_per_fov.index[meta_per_fov["inner"]]]
X = features_subset[:,features_subset.var["Feature type"] != ref_ftype].X
Y = features_subset[:,features_subset.var["Feature type"] == ref_ftype].X

In [12]:
# Get the rows of Y that are not NaN
idx_exclude = np.where(np.sum(np.isnan(Y), axis=1))[0]
X = np.delete(X, idx_exclude, 0)
Y = np.delete(Y, idx_exclude, 0)

In [13]:
# Create a mapping from old to new indices after exclusion
valid_indices = np.setdiff1d(np.arange(features_subset.shape[0]),
                             idx_exclude)
old_to_new = {old: new for new, old in enumerate(valid_indices)}

meta_filtered = meta_per_fov.loc[meta_per_fov["inner"]]
meta_filtered = meta_filtered.iloc[valid_indices]

In [14]:
# Also exclude from the CV folds
cv_folds_fov_filtered = []
for train, test in cv_folds_fov:
    # Map old indices to new indices in the filtered array
    train_filtered = np.array([old_to_new[idx] for idx in train if idx not in idx_exclude])
    test_filtered = np.array([old_to_new[idx] for idx in test if idx not in idx_exclude])
    cv_folds_fov_filtered.append((train_filtered, test_filtered))
    
    # All stages should be present in both train and test
    assert len(meta_filtered.iloc[test_filtered]["Stage"].unique()) == 4
    assert len(meta_filtered.iloc[train_filtered]["Stage"].unique()) == 4
    # Print the first test FOVs
    print(meta_filtered.iloc[test_filtered].index[:5])

Index(['A1e', 'A1n', 'A3b', 'A3e', 'A3h'], dtype='object', name='fov')
Index(['A1c', 'A1m', 'A1o', 'A1p', 'A1q'], dtype='object', name='fov')
Index(['A1l', 'A2c', 'A2l', 'A2p', 'A3l'], dtype='object', name='fov')
Index(['A1d', 'A1i', 'A1k', 'A2d', 'A2e'], dtype='object', name='fov')


In [15]:
# Visualize how many samples were excluded
print([(x.shape, y.shape) for x, y in cv_folds_fov])
print([(x.shape, y.shape) for x, y in cv_folds_fov_filtered])

[((251,), (83,)), ((249,), (85,)), ((250,), (84,)), ((252,), (82,))]
[((226,), (68,)), ((221,), (73,)), ((219,), (75,)), ((216,), (78,))]


In [16]:
clf = XGBRegressor(eval_metric='aucpr',
                   n_estimators=250, 
                   max_depth=3, 
                   device="cuda", 
                   random_state=0,
                   tree_method="hist")
clf.fit(X, Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric='aucpr', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=250, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

In [17]:
scores = cross_val_score(
    XGBRegressor(
                   objective="reg:squarederror",
                   n_estimators=250, 
                   max_depth=3, 
                   device="cuda", 
                   random_state=0,
                   tree_method="hist"),
    X,
    Y,
    cv=cv_folds_fov_filtered,
    scoring="neg_median_absolute_error"
    )
pd.Series(scores).describe()

/home/loan/miniforge3/envs/pam-keras3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:22:27] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


count    4.000000
mean    -0.372298
std      0.064366
min     -0.448136
25%     -0.396961
50%     -0.375046
75%     -0.350382
max     -0.290965
dtype: float64

In [18]:
scores = cross_val_score(
    XGBRegressor(
                   objective="reg:absoluteerror",
                   n_estimators=250, 
                   max_depth=3, 
                   device="cuda", 
                   random_state=0,
                   tree_method="hist"),
    X,
    Y,
    cv=cv_folds_fov_filtered,
    scoring="neg_median_absolute_error"
    )
pd.Series(scores).describe()

count    4.000000
mean    -0.334189
std      0.051218
min     -0.377506
25%     -0.369702
50%     -0.347419
75%     -0.311906
max     -0.264413
dtype: float64

In [19]:
scores = cross_val_score(
    XGBRegressor(  
                   objective="reg:squarederror",
                   n_estimators=250, 
                   max_depth=3, 
                   random_state=0,
                   tree_method="hist", 
                   multi_strategy="multi_output_tree"),
    X,
    Y,
    cv=cv_folds_fov_filtered,
    scoring="r2"
    )
pd.Series(scores).describe()

count    4.000000
mean     0.134023
std      0.125495
min      0.000715
25%      0.046642
50%      0.129360
75%      0.216741
max      0.276656
dtype: float64

In [20]:
for ftype in features_subset.var["Feature type"].unique():
    Xc = features_subset[:,features_subset.var["Feature type"] == ftype].X
    Xc = np.delete(Xc, idx_exclude, 0)

    scores = cross_val_score(
        XGBRegressor(  
                    objective="reg:squarederror",
                    n_estimators=250, 
                    max_depth=3, 
                    random_state=0,
                    tree_method="hist", 
                    device="cuda", 
                    ),
        Xc,
        Y,
        cv=cv_folds_fov_filtered,
        scoring="r2"
        )
    print(ftype, np.median(scores))

Metabolism 0.9836412893110675
Abundance 0.04051502432356721
Function -0.3365420823434977
Spatial -0.1483544506271177
Morphology -0.17102214514883415


In [21]:
for ftype in features_subset.var["Feature type"].unique():
    Xc = features_subset[:,features_subset.var["Feature type"] == ftype].X
    Xc = np.delete(Xc, idx_exclude, 0)

    scores = cross_val_score(
        XGBRegressor(  
                    objective="reg:absoluteerror",
                    n_estimators=250, 
                    max_depth=3, 
                    random_state=0,
                    tree_method="hist", 
                    device="cuda", 
                    ),
        Xc,
        Y,
        cv=cv_folds_fov_filtered,
        scoring="r2"
        )
    print(ftype, np.median(scores))

Metabolism 0.972981835232862
Abundance 0.09919841991687389
Function -0.25136696088067556
Spatial -0.0019724151965391096
Morphology -0.22539081617354023


In [22]:
for ftype in features_subset.var["Feature type"].unique():
    Xc = features_subset[:,features_subset.var["Feature type"] == ftype].X
    Xc = np.delete(Xc, idx_exclude, 0)

    scores = cross_val_score(
        XGBRegressor(  
                    objective="reg:squarederror",
                    n_estimators=250, 
                    max_depth=3, 
                    random_state=0,
                    tree_method="hist", 
                    multi_strategy="multi_output_tree"),
        Xc,
        Y,
        cv=cv_folds_fov_filtered,
        scoring="r2"
        )
    print(ftype, np.median(scores))

Metabolism 0.9345854837033776
Abundance 0.0732026171555761
Function -0.26570654886528566
Spatial -0.1385994374661316
Morphology -0.1623347328411497


In [23]:
for ref_ftype in features_subset.var["Feature type"].unique():
    features_subset = features[meta_per_fov.index[meta_per_fov["inner"]]]
    X = features_subset[:,features_subset.var["Feature type"] != ref_ftype].X
    Y = features_subset[:,features_subset.var["Feature type"] == ref_ftype].X

    # Get the rows of Y that are not NaN
    idx_exclude = np.where(np.sum(np.isnan(Y), axis=1))[0]
    X = np.delete(X, idx_exclude, 0)
    Y = np.delete(Y, idx_exclude, 0)

    # Create a mapping from old to new indices after exclusion
    valid_indices = np.setdiff1d(np.arange(features_subset.shape[0]),
                                idx_exclude)
    old_to_new = {old: new for new, old in enumerate(valid_indices)}

    meta_filtered = meta_per_fov.loc[meta_per_fov["inner"]]
    meta_filtered = meta_filtered.iloc[valid_indices]

    # Also exclude from the CV folds
    cv_folds_fov_filtered = []
    for train, test in cv_folds_fov:
        # Map old indices to new indices in the filtered array
        train_filtered = np.array([old_to_new[idx] for idx in train if idx not in idx_exclude])
        test_filtered = np.array([old_to_new[idx] for idx in test if idx not in idx_exclude])
        cv_folds_fov_filtered.append((train_filtered, test_filtered))
        
        # All stages should be present in both train and test
        assert len(meta_filtered.iloc[test_filtered]["Stage"].unique()) == 4
        assert len(meta_filtered.iloc[train_filtered]["Stage"].unique()) == 4
        # Print the first test FOVs
        # print(meta_filtered.iloc[test_filtered].index[:5])

    for ftype in features_subset.var["Feature type"].unique():
        Xc = features_subset[:,features_subset.var["Feature type"] == ftype].X
        Xc = np.delete(Xc, idx_exclude, 0)

        scores = cross_val_score(
            XGBRegressor(  
                        objective="reg:squarederror",
                        n_estimators=250, 
                        max_depth=3, 
                        random_state=0,
                        tree_method="hist", 
                        multi_strategy="multi_output_tree"),
            Xc,
            Y,
            cv=cv_folds_fov_filtered,
            scoring="r2"
            )
        if (np.median(scores) > 0.15) and (ref_ftype != ftype):
            print(ref_ftype, ftype, np.median(scores))

In [24]:
for ref_ftype in features_subset.var["Feature type"].unique():
    features_subset = features[meta_per_fov.index[meta_per_fov["inner"]]]
    X = features_subset[:,features_subset.var["Feature type"] != ref_ftype].X
    Y = features_subset[:,features_subset.var["Feature type"] == ref_ftype].X

    # Get the rows of Y that are not NaN
    idx_exclude = np.where(np.sum(np.isnan(Y), axis=1))[0]
    X = np.delete(X, idx_exclude, 0)
    Y = np.delete(Y, idx_exclude, 0)

    # Create a mapping from old to new indices after exclusion
    valid_indices = np.setdiff1d(np.arange(features_subset.shape[0]),
                                idx_exclude)
    old_to_new = {old: new for new, old in enumerate(valid_indices)}

    meta_filtered = meta_per_fov.loc[meta_per_fov["inner"]]
    meta_filtered = meta_filtered.iloc[valid_indices]

    # Also exclude from the CV folds
    cv_folds_fov_filtered = []
    for train, test in cv_folds_fov:
        # Map old indices to new indices in the filtered array
        train_filtered = np.array([old_to_new[idx] for idx in train if idx not in idx_exclude])
        test_filtered = np.array([old_to_new[idx] for idx in test if idx not in idx_exclude])
        cv_folds_fov_filtered.append((train_filtered, test_filtered))
        
        # All stages should be present in both train and test
        assert len(meta_filtered.iloc[test_filtered]["Stage"].unique()) == 4
        assert len(meta_filtered.iloc[train_filtered]["Stage"].unique()) == 4
        # Print the first test FOVs
        # print(meta_filtered.iloc[test_filtered].index[:5])

    for ftype in features_subset.var["Feature type"].unique():
        if ftype == ref_ftype:
            continue
        Xc = features_subset[:,features_subset.var["Feature type"] == ftype].X
        Xc = np.delete(Xc, idx_exclude, 0)

        scores = cross_val_score(
            XGBRegressor(  
                        objective="reg:squarederror",
                        n_estimators=250, 
                        max_depth=3, 
                        random_state=0,
                        tree_method="hist", 
                        device="cuda"),
            Xc,
            Y,
            cv=cv_folds_fov_filtered,
            scoring="r2"
            )
        if (np.median(scores) > 0.15) and (ref_ftype != ftype):
            print(ref_ftype, ftype, np.median(scores))

In [25]:
for ref_ftype in features_subset.var["Feature type"].unique():
    features_subset = features[meta_per_fov.index[meta_per_fov["inner"]]]
    X = features_subset[:,features_subset.var["Feature type"] != ref_ftype].X
    Y = features_subset[:,features_subset.var["Feature type"] == ref_ftype].X

    # Get the rows of Y that are not NaN
    idx_exclude = np.where(np.sum(np.isnan(Y), axis=1))[0]
    X = np.delete(X, idx_exclude, 0)
    Y = np.delete(Y, idx_exclude, 0)

    # Create a mapping from old to new indices after exclusion
    valid_indices = np.setdiff1d(np.arange(features_subset.shape[0]),
                                idx_exclude)
    old_to_new = {old: new for new, old in enumerate(valid_indices)}

    meta_filtered = meta_per_fov.loc[meta_per_fov["inner"]]
    meta_filtered = meta_filtered.iloc[valid_indices]

    # Also exclude from the CV folds
    cv_folds_fov_filtered = []
    for train, test in cv_folds_fov:
        # Map old indices to new indices in the filtered array
        train_filtered = np.array([old_to_new[idx] for idx in train if idx not in idx_exclude])
        test_filtered = np.array([old_to_new[idx] for idx in test if idx not in idx_exclude])
        cv_folds_fov_filtered.append((train_filtered, test_filtered))
        
        # All stages should be present in both train and test
        assert len(meta_filtered.iloc[test_filtered]["Stage"].unique()) == 4
        assert len(meta_filtered.iloc[train_filtered]["Stage"].unique()) == 4
        # Print the first test FOVs
        # print(meta_filtered.iloc[test_filtered].index[:5])

    for ftype in features_subset.var["Feature type"].unique():
        if ftype == ref_ftype:
            continue
        Xc = features_subset[:,features_subset.var["Feature type"] == ftype].X
        Xc = np.delete(Xc, idx_exclude, 0)

        scores = cross_val_score(
            XGBRegressor(  
                        objective="reg:absoluteerror",
                        n_estimators=250, 
                        max_depth=3, 
                        random_state=0,
                        tree_method="hist", 
                        device="cuda"),
            Xc,
            Y,
            cv=cv_folds_fov_filtered,
            scoring="r2"
            )
        if (np.median(scores) > 0.15) and (ref_ftype != ftype):
            print(ref_ftype, ftype, np.median(scores))

## Main model selection loop

In [26]:
def process_fold(X, Y, train, test, n_estimators = 250, n_depth = 3):
    # logging.getLogger('tensorflow').setLevel(logging.ERROR)
    # with warnings.catch_warnings(action="ignore"):      
    # Step 1: Define train and test subsets of the compositional data
    train_func = X[train]
    test_func = X[test]
    train_meta = Y[train]
    test_meta = Y[test]

    # Step 2: Train a classifier on the training data composition to predict the stage of each FOV
    xgb = XGBRegressor(  
                    objective="reg:absoluteerror",
                    n_estimators=n_estimators, 
                    max_depth=n_depth, 
                    random_state=0,
                    tree_method="hist", 
                    device="cuda")
    xgb.fit(train_func,
            train_meta)

    # Step 3: Predict stage of each FOV in the test data
    preds = xgb.predict(test_func)

    # Step 4: Compute f1_score
    score = r2_score(test_meta, 
                    preds)
    return score

In [27]:
def test_parameters(X, Y, cv_folds, file_path, estimators, depth, verbose = False):
    scores = [0 for _ in cv_folds]
    for i, (train, test) in enumerate(cv_folds):
        scores[i] = process_fold(X, Y, train, test, n_estimators=estimators, n_depth=depth)
        if scores[i] <= 0:
            if verbose:
                print("Bad performance, skipping.", file_path, estimators, depth)
            return -1
    mean_score = np.median(scores)
    # Append to f1 score log file
    with open(file_path, "a") as f:
        f.write(f"{estimators},{depth},{mean_score}\n")
    
    if verbose:
        print(file_path, estimators, depth, mean_score)
    return mean_score

In [28]:
process_fold(Xc, Y, train_filtered, test_filtered)

-0.2901322015354101

In [29]:
test_parameters(Xc, Y, cv_folds_fov_filtered, "../../data/ignoreme.txt", estimators = 250, depth = 3, verbose = True)

Bad performance, skipping. ../../data/ignoreme.txt 250 3


-1

In [30]:
from joblib import Parallel, delayed
import warnings, logging

def test_celltype_pair(ref_ftype, query_ftype):
    X = features_subset[:,features_subset.var["Feature type"] != ref_ftype].X
    Y = features_subset[:,features_subset.var["Feature type"] == ref_ftype].X

    # Get the rows of Y that are not NaN
    idx_exclude = np.where(np.sum(np.isnan(Y), axis=1))[0]
    X = np.delete(X, idx_exclude, 0)
    Y = np.delete(Y, idx_exclude, 0)

    # Create a mapping from old to new indices after exclusion
    valid_indices = np.setdiff1d(np.arange(features_subset.shape[0]),
                                idx_exclude)
    old_to_new = {old: new for new, old in enumerate(valid_indices)}

    meta_filtered = meta_per_fov.loc[meta_per_fov["inner"]]
    meta_filtered = meta_filtered.iloc[valid_indices]

    # Also exclude from the CV folds
    cv_folds_fov_filtered = []
    for train, test in cv_folds_fov:
        # Map old indices to new indices in the filtered array
        train_filtered = np.array([old_to_new[idx] for idx in train if idx not in idx_exclude])
        test_filtered = np.array([old_to_new[idx] for idx in test if idx not in idx_exclude])
        cv_folds_fov_filtered.append((train_filtered, test_filtered))
        
        # All stages should be present in both train and test
        assert len(meta_filtered.iloc[test_filtered]["Stage"].unique()) == 4
        assert len(meta_filtered.iloc[train_filtered]["Stage"].unique()) == 4

    Xc = features_subset[:,features_subset.var["Feature type"] == query_ftype].X
    Xc = np.delete(Xc, idx_exclude, 0)

    # Loop over parameters
    estimator_range = np.arange(15, 511, 10)
    depth_range = [2,3,4,5]
    estimator_range, depth_range = np.meshgrid(estimator_range, depth_range)
    Parallel(n_jobs=28)(delayed(test_parameters)(Xc, Y, cv_folds_fov_filtered, 
                    f"../../data/model_featuretypes_{ref_ftype}_{query_ftype}.csv",
                     estimators = r_estimators, depth = r_depth, 
                     verbose = False)
                    for r_estimators, r_depth in zip(estimator_range.ravel(), depth_range.ravel()))
    

In [31]:
for f1 in features.var["Feature type"].unique():
    for f2 in features.var["Feature type"].unique():
        # if f1 == "Metabolism" and f2 == "Function":
        #     continue # Skip pre-computed pairs
        if f1 != f2:
            print(f"Processing pair: {f1} vs {f2}")
            test_celltype_pair(f1, f2)

Processing pair: Metabolism vs Abundance


/home/loan/miniforge3/envs/pam-keras3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:36:12] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/loan/miniforge3/envs/pam-keras3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:36:12] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the d

Processing pair: Metabolism vs Spatial
Processing pair: Metabolism vs Morphology
Processing pair: Abundance vs Metabolism
Processing pair: Abundance vs Function
Processing pair: Abundance vs Spatial
Processing pair: Abundance vs Morphology
Processing pair: Function vs Metabolism
Processing pair: Function vs Abundance
Processing pair: Function vs Spatial
Processing pair: Function vs Morphology
Processing pair: Spatial vs Metabolism
Processing pair: Spatial vs Abundance
Processing pair: Spatial vs Function
Processing pair: Spatial vs Morphology
Processing pair: Morphology vs Metabolism
Processing pair: Morphology vs Abundance
Processing pair: Morphology vs Function
Processing pair: Morphology vs Spatial


In [33]:
test_celltype_pair("Metabolism", "Function")

/home/loan/miniforge3/envs/pam-keras3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:15:50] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/loan/miniforge3/envs/pam-keras3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:16:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the d

## Validate best model for each pair

In [34]:
def validate_celltype_pair(ref_ftype, query_ftype, max_depth, max_estimators):
    X_train = features[meta_per_fov.index[meta_per_fov["inner"]],
                   features_subset.var["Feature type"] == query_ftype]
    Y_train = features[meta_per_fov.index[meta_per_fov["inner"]],
                    features_subset.var["Feature type"] == ref_ftype]
    
    # Get the rows of Y_train that are not NaN
    idx_exclude = np.where(np.sum(np.isnan(Y_train.X), axis=1))[0]
    X_train = np.delete(X_train.X, idx_exclude, 0)
    Y_train = np.delete(Y_train.X, idx_exclude, 0)

    # Prepare validation data
    X_val = features[meta_per_fov.index[~meta_per_fov["inner"]], 
                     features.var["Feature type"] == query_ftype].X
    Y_val = features[meta_per_fov.index[~meta_per_fov["inner"]], 
                     features.var["Feature type"] == ref_ftype].X
    
    # Exclude rows with NaN in Y_val
    idx_exclude_val = np.where(np.sum(np.isnan(Y_val), axis=1))[0]
    X_val = np.delete(X_val, idx_exclude_val, 0)
    Y_val = np.delete(Y_val, idx_exclude_val, 0)

    xgb = XGBRegressor(  
                    objective="reg:absoluteerror",
                    n_estimators=max_estimators, 
                    max_depth=max_depth, 
                    random_state=0,
                    tree_method="hist", 
                    device="cuda")
    xgb.fit(X_train,
            Y_train)

    # Predict stage of each FOV in the validation data
    preds = xgb.predict(X_val)

    # Compute r2 score
    score = r2_score(Y_val, 
                    preds)
    return score

In [35]:
def validate_celltype_pair_stage_strafied(ref_ftype, query_ftype, max_depth, max_estimators):
    X_train = features[meta_per_fov.index[meta_per_fov["inner"]],
                   features_subset.var["Feature type"] == query_ftype]
    Y_train = features[meta_per_fov.index[meta_per_fov["inner"]],
                    features_subset.var["Feature type"] == ref_ftype]
    
    # Get the rows of Y_train that are not NaN
    idx_exclude = np.where(np.sum(np.isnan(Y_train.X), axis=1))[0]
    X_train = np.delete(X_train.X, idx_exclude, 0)
    Y_train = np.delete(Y_train.X, idx_exclude, 0)

    # Prepare validation data
    X_val = features[meta_per_fov.index[~meta_per_fov["inner"]], 
                     features.var["Feature type"] == query_ftype].X
    Y_val = features[meta_per_fov.index[~meta_per_fov["inner"]], 
                     features.var["Feature type"] == ref_ftype].X
    
    # Exclude rows with NaN in Y_val
    idx_exclude_val = np.where(np.sum(np.isnan(Y_val), axis=1))[0]
    X_val = np.delete(X_val, idx_exclude_val, 0)
    Y_val = np.delete(Y_val, idx_exclude_val, 0)

    xgb = XGBRegressor(  
                    objective="reg:absoluteerror",
                    n_estimators=max_estimators, 
                    max_depth=max_depth, 
                    random_state=0,
                    tree_method="hist", 
                    device="cuda")
    xgb.fit(X_train,
            Y_train)

    # Predict stage of each FOV in the validation data
    preds = xgb.predict(X_val)

    # Compute r2 score
    score = r2_score(Y_val, 
                    preds)

    # Update meta for selected validation set
    meta_val = meta_per_fov.loc[~meta_per_fov["inner"]]
    
    if len(idx_exclude_val):
        meta_val = meta_val.drop(meta_val.index[idx_exclude_val])

    # Validation on ealy stages only
    X_early = X_val[meta_val["Stage"].isin(["pT1", "pT2"])]
    Y_early = Y_val[meta_val["Stage"].isin(["pT1", "pT2"])]

    preds_early = xgb.predict(X_early)
    score_early = r2_score(Y_early, 
                           preds_early)
    
    # Validation on late stages only
    X_late = X_val[meta_val["Stage"].isin(["pT3", "pT4"])]
    Y_late = Y_val[meta_val["Stage"].isin(["pT3", "pT4"])]

    preds_late = xgb.predict(X_late)
    score_late = r2_score(Y_late, 
                          preds_late)
    
    return score, score_early, score_late

In [36]:
data_files = os.listdir("../../data")
r2_threshold = 0.2 # Ignore models with low performance on CV inner folds

# Create a directed weighted graph to store the predictability between feature types
pred_network = nx.DiGraph()
pred_network_early = nx.DiGraph()
pred_network_late = nx.DiGraph()

for f1 in features.var["Feature type"].unique():
    for f2 in features.var["Feature type"].unique():
        model_file = f"model_featuretypes_{f1}_{f2}.csv"
        # Some pairs were skipped due to bad performance
        if model_file in data_files:
            pairformance = pd.read_csv(f"../../data/{model_file}", header=None, 
                                       names=["n_estimators", "max_depth", "mean_r2_score"])
            pairformance = pairformance.sort_values("mean_r2_score", ascending=False)
            best_model = pairformance.iloc[0]
            print(f"Best model for {f1} vs {f2}: n_estimators = {best_model['n_estimators']}, "
                  f"max_depth = {best_model['max_depth']}, "
                  f"median r2 score = {best_model['mean_r2_score']:.4f}")
            if best_model['mean_r2_score'] > r2_threshold:
                # Validate the best model on the outer validation set
                scores = validate_celltype_pair_stage_strafied(f1, f2,
                                                  int(best_model['max_depth']),
                                                  int(best_model['n_estimators']))
                print(f"Validation r2 score for {f1} vs {f2}: {scores[0]:.4f}")
                if scores[0] > r2_threshold:
                    # Add an edge to the graph with the r2 score as weight
                    pred_network.add_edge(f2, f1, weight=scores[0]) # f2 predicts f1
                if scores[1] > r2_threshold:
                    # Add an edge to the early stages graph with the r2 score as weight
                    pred_network_early.add_edge(f2, f1, weight=scores[1])
                if scores[2] > r2_threshold:
                    # Add an edge to the late stages graph with the r2 score as weight
                    pred_network_late.add_edge(f2, f1, weight=scores[2])

# Save the networks to a GML files
nx.write_gml(pred_network, "../../data/featuretype_graph.gml")
nx.write_gml(pred_network_early, "../../data/featuretype_graph_early.gml")
nx.write_gml(pred_network_late, "../../data/featuretype_graph_late.gml")

Best model for Metabolism vs Abundance: n_estimators = 25.0, max_depth = 2.0, median r2 score = 0.1077
Best model for Abundance vs Spatial: n_estimators = 15.0, max_depth = 2.0, median r2 score = 0.1106


In [37]:
print(pred_network.edges, 
    pred_network_early.edges,
    pred_network_late.edges)

[] [] []
